### Integrating with Apache Airflow
**Description**: Integrate Great Expectations with Apache Airflow to run data quality checks automatically in your DAG.

**Steps**:
1. Install Airflow (if you haven't already):
2. Airflow DAG Integration:
    - Create a DAG file:
3. Deploy and Test:
    - Place this file in your Airflow DAGs directory and start your Airflow scheduler.
    - Open the Airflow UI and trigger the DAG to see it run your expectations.

In [2]:
# Write your code from here
!pip install "apache-airflow==2.7.2" --constraint "https://raw.githubusercontent.com/apache/airflow/constraints-2.7.2/constraints-3.8.txt"
!pip install great_expectations

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 26.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 27.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.5/516.5 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.3/83.3 kB 18

In [3]:
from datetime import datetime
from airflow import DAG
from airflow.operators.python import PythonOperator
import great_expectations as gx
import pandas as pd

def validate_data():
    df = pd.DataFrame({
        "customer_id": [1, 2, 3],
        "purchase_amount": [120.5, 89.0, 300.75],
        "age": [25, 40, 60]
    })

    context = gx.get_context()

    suite = context.add_or_update_expectation_suite("customer_suite")

    validator = context.sources.pandas_default.read_dataframe(df)
    validator.expect_column_to_exist("customer_id")
    validator.expect_column_values_to_be_of_type("purchase_amount", "FLOAT")
    validator.expect_column_values_to_be_between("age", min_value=18, max_value=65)

    validator.save_expectation_suite(discard_failed_expectations=False)

    checkpoint = context.add_or_update_checkpoint(
        name="customer_checkpoint",
        validator=validator,
    )

    result = checkpoint.run()
    assert result["success"], "Data validation failed!"

with DAG(
    dag_id="gx_airflow_validation",
    start_date=datetime(2024, 1, 1),
    schedule_interval="@daily",
    catchup=False,
) as dag:
    task = PythonOperator(
        task_id="run_validation",
        python_callable=validate_data
    )








[2025-05-20T15:47:16.973+0000] {warnings.py:109} WARNING - /tmp/ipykernel_52191/3636649549.py:33: RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead. 
  with DAG(

